<a href="https://colab.research.google.com/github/shimels1/node-assessment/blob/master/CSL_progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import cv2
import time
import numpy
import csv
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
import os
import matplotlib.pyplot as plt
import csv
from keras.preprocessing import image
from keras.models import Sequential
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from matplotlib import pyplot as plt
import numpy as np
import numpy
from PIL import Image
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
import keras
import sys
# keras imports
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Input
import csv
from keras.preprocessing import image
# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
#import glob
# import cv2
import h5py
import os
#import json
import datetime
import time
import effTools as eff
import sys
import os

In [12]:
face_cascades = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [13]:
def median_filter(data, filter_size):
    temp = []
    indexer = filter_size // 2
    data_final = []
    data_final = numpy.zeros((len(data),len(data[0])))
    for i in range(len(data)):

        for j in range(len(data[0])):

            for z in range(filter_size):
                if i + z - indexer < 0 or i + z - indexer > len(data) - 1:
                    for c in range(filter_size):
                        temp.append(0)
                else:
                    if j + z - indexer < 0 or j + indexer > len(data[0]) - 1:
                        temp.append(0)
                    else:
                        for k in range(filter_size):
                            temp.append(data[i + z - indexer][j + k - indexer])

            temp.sort()
            data_final[i][j] = temp[len(temp) // 2]
            temp = []
    return data_final

In [14]:
# store all file name in the folder into one array list
dataFileName = []

for filename in os.listdir("/content/drive/MyDrive/Adjectives/99. healthy"):
  if filename.endswith("MOV"): 
      dataFileName.append(filename)

In [15]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn = Sequential()

# Adding a 1nd convolutional layer
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu', input_shape=[224, 224, 3]))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

# Adding a 2nd convolutional layer
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

# # Adding a 3rd convolutional layer
# cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
# cnn.add(MaxPooling2D(pool_size=2, strides=2))

# # Adding a 4rd convolutional layer
# cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
# cnn.add(MaxPooling2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(Flatten())

# cnn.add(Dense(64, activation='relu'))
# # cnn.add(Dropout(0.3))

# cnn.add(Dense(64, activation='relu'))
# # cnn.add(Dropout(0.3))       
# cnn.add(Dense(3, activation='softmax'))

# cnn.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
# cnn.summary()

In [ ]:
# image preprocessing (resize image, nose removale using midian filter, feauter exstraction using cnn)
for i in range(0,len(dataFileName)):
    cap = cv2.VideoCapture("drive/MyDrive/Adjectives/99. healthy/"+dataFileName[i])
    if cap.isOpened() == False:
        print("Error File Not Found")
    
    c = 0
  
    while cap.isOpened():
      success, img = cap.read()
      # print(i)
      if success == True:
        
        #save the result
        orginalImage = cv2.resize(img, (224, 224))
        cv2.imwrite("a.jpg",orginalImage)

        # Face detection and draw white background
        gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascades.detectMultiScale(gray_frame, 1.3, 5)

        for (x, y, w, h) in faces:

            # draw red box
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), -1)
            roi_gray = gray_frame[y:y + h, x:x + w]
            roi_color = img[y:y + h, x:x + w]

            #save the result  
            FaceDetectionWithRedBackground = cv2.resize(img, (224, 224))
            cv2.imwrite("b.jpg",FaceDetectionWithRedBackground)

            # converting from gbr to hsv color space
            img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            # skin color range for hsv color space
            HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17, 170, 255))
            HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, numpy.ones((3, 3), numpy.uint8))

            # converting from gbr to YCbCr color space
            img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
            # skin color range for hsv color space
            YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255, 180, 135))
            YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, numpy.ones((3, 3), numpy.uint8))

            # merge skin detection (YCbCr and hsv)
            global_mask = cv2.bitwise_and(YCrCb_mask, HSV_mask)
            global_mask = cv2.medianBlur(global_mask, 3)
            global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, numpy.ones((4, 4), numpy.uint8))

            HSV_result = cv2.bitwise_not(HSV_mask)
            YCrCb_result = cv2.bitwise_not(YCrCb_mask)
            global_result = cv2.bitwise_not(global_mask)

            # # MedianFilter
            # arr = numpy.array(global_result)
            # removed_noise = median_filter(arr, 3)
            # global_result = np.asarray(removed_noise)

            if c < 50:
              global_result = cv2.resize(global_result, (224, 224))
              cv2.imwrite("3_global_resultm.jpg",global_result)
              clear_output(wait=True)
              # display the result
              orginalImage=image.load_img('a.jpg', target_size = (224, 224))
              FaceDetectionWithRedBackground=image.load_img('b.jpg', target_size = (224, 224))
              FaceDetectionWithRedBackground = image.img_to_array(FaceDetectionWithRedBackground)
              orginalImage = image.img_to_array(orginalImage)
              cv2_imshow(orginalImage)
              cv2_imshow(FaceDetectionWithRedBackground)
              cv2_imshow(global_result)

              test_image=image.load_img('3_global_resultm.jpg', target_size = (224, 224))
              test_image = image.img_to_array(test_image)
              test_image = np.expand_dims(test_image, axis = 0)
              r1=cnn.predict(test_image)
              a=np.array(r1[0])
              a = numpy.append(a, "healthy")
              with open('/content/drive/MyDrive/testdataset6.csv', 'a+', newline='') as f:
                write = csv.writer(f)
                write.writerow(a)
              cv2.waitKey(1) & 0xff
            
            c = c + 1
      else:
          break

In [17]:
# Recurrent Neural Network

# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# # Importing the training set
dataset_train = pd.read_csv('/content/drive/MyDrive/testdataset6.csv')
print(dataset_train.shape)

(31, 93313)


In [30]:
training_set = dataset_train.iloc[:, 0:93312].values
training_set_y = dataset_train.iloc[:, 93312:93313].values

training_test = dataset_train.iloc[:, 0:93312].values
training_test_y = dataset_train.iloc[:, 93312:93313].values
print(training_test)

[[18.674448  0.       38.18519  ...  4.015486 57.176132  0.      ]
 [18.674448  0.       38.18519  ...  4.015486 57.176132  0.      ]
 [18.674448  0.       38.18519  ...  4.015486 57.176132  0.      ]
 ...
 [18.674448  0.       38.18519  ...  4.015486 57.176132  0.      ]
 [18.674448  0.       38.18519  ...  4.015486 57.176132  0.      ]
 [18.674448  0.       38.18519  ...  4.015486 57.176132  0.      ]]


In [31]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)


In [32]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_test_scaled = sc.fit_transform(training_test)

In [ ]:
c=0
k=0
wordArry = []
X = []
y = []
t_X = []
t_y = []
dry = 0
sick = 0
healthy = 0
for i in range(0,len(training_set_scaled)):
  c = c + 1
  wordArry.append(training_set_scaled[i])
  if ( c == 50 ):
    if training_set_y[i] == 'dry':
      if dry < 14:
        X.append(wordArry)
        y.append(0)
        dry = dry + 1
      else:
        t_X.append(wordArry)
        t_y.append(0)
    if training_set_y[i] == 'sick':
      if sick < 14:
        X.append(wordArry)
        y.append(1)
        sick = sick + 1
      else:
        t_X.append(wordArry)
        t_y.append(0)
    if training_set_y[i] == 'healthy':
      if healthy < 14:
        X.append(wordArry)
        y.append(2)
        healthy = healthy + 1
      else:
        t_X.append(wordArry)
        t_y.append(0)
    wordArry = []
    k = k + 1
    c = 0
print(dry,
sick,
healthy)
X_train = np.asarray(X, dtype=np.float32)
y_train = np.asarray(y, dtype=np.float32)
print(X_train.shape)
print(y_train.shape)
X_test = np.asarray(t_X, dtype=np.float32)
y_test = np.asarray(t_y, dtype=np.float32)
print(X_test.shape)
print(y_test.shape)

In [ ]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train).astype(int)

In [ ]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

In [ ]:
# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50,  activation='relu', return_sequences = True, input_shape = (50, 6272)))
regressor.add(Dropout(0.2))

In [ ]:
# # Adding a second LSTM layer and some Dropout regularisation
# regressor.add(LSTM(units = 50, return_sequences = True, activation='relu'))
# regressor.add(Dropout(0.2))

# # Adding a third LSTM layer and some Dropout regularisation
# regressor.add(LSTM(units = 50, return_sequences = True, activation='relu'))
# regressor.add(Dropout(0.2))

# # Adding a fourth LSTM layer and some Dropout regularisation
# regressor.add(LSTM(units = 50))
# regressor.add(Dropout(0.2))

# Adding the output layer
# regressor.add(Dense(units = 1))
regressor.add(Dense(3, activation='softmax'))

# Compiling the RNN
regressor.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# regressor.summary()

# Fitting the RNN to the Training set
cnn.fit(X_train, y_train, epochs = 150, batch_size = 32)

In [ ]:

xxxx = np.array(X_test)
predict = regressor.predict(xxxx)
a2 = 0
a3 = 0
a4 = 0
for i in range(12):
  if(0 == np.argmax(predict[i])):
    a2 = a2 + 1
  if(1 == np.argmax(predict[i])):
    a3 = a3 + 1
  if(2 == np.argmax(predict[i])):
    a4 = a4 + 1
print(a2+a3+a4)
print(a2,a3,a4)